In [12]:
import numpy as np
import os
import pandas as pd
import math
import re
from itertools import combinations, product


In [2]:
### process an inputted formula to determine the anions and cations

# information inputted by user for Cs2AgBiCl6
A1 = 'Cs'
A2 = A1
B1 = 'Ag'
B2 = 'Bi'
X1 = 'Cl'
X2 = X1

As = list(set([A1, A2]))
Bs = list(set([B1, B2]))
Xs = list(set([X1, X2]))

if len(As) == 1:
    A_piece = ''.join([As[0], '2'])
else:
    A_piece = ''.join(As)
if len(Bs) == 1:
    B_piece = ''.join([Bs[0], '2'])
else:
    B_piece = ''.join(Bs)
if len(Xs) == 1:
    X_piece = ''.join([Xs[0], '6'])
else:
    X_piece = ''.join([Xs[0], '3', Xs[1], '3'])
formula = ''.join([A_piece, B_piece, X_piece])

print('Based on the user inputs, the formula = %s' 
      % formula)

el_num_pairs = re.findall('([A-Z][a-z]\d*)|([A-Z]\d*)', formula)
el_num_pairs = [[pair[idx] for idx in range(len(pair))if pair[idx] != ''][0] for pair in el_num_pairs]
el_num_pairs = [pair+'1' if bool(re.search(re.compile('\d'), pair)) == False else pair for pair in el_num_pairs]
el_num_pairs = sorted(el_num_pairs)
good_form = ''.join(el_num_pairs)

print('A standard version of the fomrula = %s' 
      % good_form)

Based on the user inputs, the formula = Cs2AgBiCl6
A standard version of the fomrula = Ag1Bi1Cl6Cs2


In [3]:
### normalizing the concentrations to an ABX3 basis

els = re.findall('[A-Z][a-z]?', good_form)
nums = list(map(int, re.findall('\d+', good_form)))
concentrations = [num / np.sum(nums) for num in nums]
conc_dict = {els[idx] : concentrations[idx] *np.sum(nums)/2 for idx in range(len(els))}
for el in els:
    print('The stoichiometric weight of each element is %s = %s' % (el, conc_dict[el]))


The stoichiometric weight of each element is Ag = 0.5
The stoichiometric weight of each element is Bi = 0.5
The stoichiometric weight of each element is Cl = 3.0
The stoichiometric weight of each element is Cs = 1.0


In [4]:
### define some common oxidation states

# oxidation states if the element is the anion
X_ox_dict = {'N' : -3,
             'O' : -2,
             'S' : -2,
             'Se' : -2,
             'F' : -1,
             'Cl' : -1,
             'Br' : -1,
             'I' : -1}

# common cation oxidation states
plus_one = ['H', 'Li', 'Na', 'K', 'Rb', 'Cs', 'Fr', 'Ag']
plus_two = ['Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra']
plus_three = ['Sc', 'Y', 'La', 'Al', 'Ga', 'In',
              'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb',
              'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']

In [5]:
### make dictionary of Shannon ionic radii

# starting with table available at v.web.umkc.edu/vanhornj/Radii.xls with Sn2+ added from 10.1039/c5sc04845a
# and organic cations from 10.1039/C4SC02211D
df = pd.read_csv('Shannon_Effective_Ionic_Radii.csv')

df = df.rename(columns = {'OX. State': 'ox',
                          'Coord. #': 'coord',
                          'Crystal Radius': 'rcryst',
                          'Ionic Radius': 'rion',
                          'Spin State' : 'spin'})
    
df['spin'] = [spin if spin in ['HS', 'LS'] else 'only_spin' for spin in df.spin.values]

def get_el(row):
    ION = row['ION']
    if ' ' in ION:
        return ION.split(' ')[0]
    elif '+' in ION:
        return ION.split('+')[0]
    elif '-' in ION:
        return ION.split('-')[0]

df['el'] = df.apply(lambda row: get_el(row), axis = 1)

# get allowed oxidation states for each ion
el_to_ox = {}
for el in df.el.values:
    el_to_ox[el] = list(set(df.ox.get((df['el'] == el)).tolist()))

# get ionic radii as function of oxidation state -> coordination number -> spin state
ionic_radii_dict = {}
for el in el_to_ox:
    # list of Shannon oxidation states for each element
    oxs = el_to_ox[el]
    ox_to_coord = {}
    for ox in oxs:
        # list of coordination numbers for each (element, oxidation state)
        coords = df.coord.get((df['el'] == el) & (df['ox'] == ox)).tolist()
        ox_to_coord[ox] = coords
        coord_to_spin = {}
        for coord in ox_to_coord[ox]:
            # list of spin states for each (element, oxidation state, coordination number)
            spin = df.spin.get((df['el'] == el) & (df['ox'] == ox) & (df['coord'] == coord)).tolist()
            coord_to_spin[coord] = spin
            spin_to_rad = {}
            for spin in coord_to_spin[coord]:
                # list of radiis for each (element, oxidation state, coordination number)
                rad = df.rion.get((df['el'] == el) & (df['ox'] == ox) & (df['coord'] == coord) & (df['spin'] == spin)).tolist()[0]
                spin_to_rad[spin] = rad  
                coord_to_spin[coord] = spin_to_rad
                ox_to_coord[ox] = coord_to_spin
    ionic_radii_dict[el] = ox_to_coord

# assign spin state for transition metals (assumes that if an ion can be high-spin, it will be)
spin_els = ['Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu']
starting_d = [4, 5, 6, 7, 8, 9]
d_dict = dict(zip(spin_els, starting_d))
for el in spin_els:
    for ox in ionic_radii_dict[el].keys():
        for coord in ionic_radii_dict[el][ox].keys():
            if len(ionic_radii_dict[el][ox][coord].keys()) > 1:
                num_d = d_dict[el] + 2 - ox
                if num_d in [4, 5, 6, 7]:
                    ionic_radii_dict[el][ox][coord]['only_spin'] = ionic_radii_dict[el][ox][coord]['HS']
                else:
                    ionic_radii_dict[el][ox][coord]['only_spin'] = ionic_radii_dict[el][ox][coord]['LS']
            elif 'HS' in ionic_radii_dict[el][ox][coord].keys():
                ionic_radii_dict[el][ox][coord]['only_spin'] = ionic_radii_dict[el][ox][coord]['HS']
            elif 'LS' in ionic_radii_dict[el][ox][coord].keys():
                ionic_radii_dict[el][ox][coord]['only_spin'] = ionic_radii_dict[el][ox][coord]['LS']

print('e.g., the ionic radius of Ti4+ at CN of 6 = %.3f Angstrom' % ionic_radii_dict['Ti'][4][6]['only_spin'])
Shannon_dict = ionic_radii_dict

e.g., the ionic radius of Ti4+ at CN of 6 = 0.605 Angstrom


In [8]:
### determine the allowed oxidation states for each element in the compound

# generate dictionary of each site in the structure (which could have unique ox state)
site_dict = site_dict = {els[idx] : ['_'.join([els[idx], str(counter)]) 
                                     for counter in range(nums[idx])] 
                                     for idx in range(len(els))}
cations = As + Bs
ox_dict = {}
for cation in cations:
    tmp_dict1 = {}
    sites = site_dict[cation]
    for site in sites:
        tmp_dict2 = {}
        if cation in plus_one:
            oxs = [1]
        elif cation in plus_two:
            oxs = [2]
        else:
            oxs = [val for val in list(Shannon_dict[cation].keys()) if val > 0]
        tmp_dict2['oxs'] = oxs
        tmp_dict1[site] = tmp_dict2
    ox_dict[cation] = tmp_dict1
for X in Xs:
    tmp_dict1 = {}
    sites = site_dict[X]
    for site in sites:
        tmp_dict2 = {}
        tmp_dict2['oxs'] = [X_ox_dict[X]]
        tmp_dict1[site] = tmp_dict2
    ox_dict[X] = tmp_dict1

for el in els:
    print('The allowed oxidation state(s) of %s = %s' % (el, ox_dict[el]))

The allowed oxidation state(s) of Ag = {'Ag_0': {'oxs': [1]}}
The allowed oxidation state(s) of Bi = {'Bi_0': {'oxs': [3, 5]}}
The allowed oxidation state(s) of Cl = {'Cl_0': {'oxs': [-1]}, 'Cl_1': {'oxs': [-1]}, 'Cl_2': {'oxs': [-1]}, 'Cl_3': {'oxs': [-1]}, 'Cl_4': {'oxs': [-1]}, 'Cl_5': {'oxs': [-1]}}
The allowed oxidation state(s) of Cs = {'Cs_0': {'oxs': [1]}, 'Cs_1': {'oxs': [1]}}


In [18]:
### find all charge-balanced cation oxidation state combinations

# compute anion charge
X_charge = 0
allowed_ox = ox_dict
for key in allowed_ox:
    if key in Xs:
        X_sites = allowed_ox[key]
        for X_site in X_sites:
            X_charge += allowed_ox[key][X_site]['oxs'][0]
            
# get dictionary of indices
idx_dict = {}
count = 0
for key in cations:
    num_sites = len(allowed_ox[key].keys())
    indices = list(np.arange(count, count + num_sites))
    count += num_sites
    idx_dict[key] = indices
            
            
lists = [allowed_ox[key][site]['oxs'] for key in cations for site in list(allowed_ox[key].keys())]
combos = list(product(*lists))
isovalent_combos = []
suitable_combos = []
for combo in combos:
    iso_count = 0
    suit_count = 0
    for key in idx_dict:
        curr_oxs = [combo[idx] for idx in idx_dict[key]]
        if np.min(curr_oxs) == np.max(curr_oxs):
            iso_count += 1
        if np.min(curr_oxs) >= np.max(curr_oxs) - 1:
            suit_count += 1
    if iso_count == len(cations):
        isovalent_combos.append(combo)
    if suit_count == len(cations):
        suitable_combos.append(combo)
bal_combos = [combo for combo in isovalent_combos if np.sum(combo) == -X_charge]
if len(bal_combos) > 0:
    combo_to_idx_ox = {}
    for combo in bal_combos:
        idx_to_ox = {}
        for key in idx_dict:
            idx_to_ox[key] = sorted([combo[idx] for idx in idx_dict[key]])
            if idx_to_ox not in list(combo_to_idx_ox.values()):
                combo_to_idx_ox[combo] = idx_to_ox                    
        combo_to_idx_ox[combo] = idx_to_ox
else:
    bal_combos = [combo for combo in suitable_combos if combo not in isovalent_combos if np.sum(combo) == -X_charge]
    combo_to_idx_ox = {}
    for combo in bal_combos:
        idx_to_ox = {}
        for key in idx_dict:
            idx_to_ox[key] = sorted([combo[idx] for idx in idx_dict[key]])
            if idx_to_ox not in list(combo_to_idx_ox.values()):
                combo_to_idx_ox[combo] = idx_to_ox

# get unique combos
combos = combo_to_idx_ox
unique_combos = {}
for combo in combos:
    if combos[combo] not in list(unique_combos.values()):
        unique_combos[combo] = combos[combo]

# get most isovalent combos
hetero_dict = {}
for combo in combos:
    sum_states = 0
    for cation in cations:
        sum_states += len(list(set(combos[combo][cation])))
    hetero_dict[combo] = sum_states - len(set(cations))
min_heterovalency = np.min(list(hetero_dict.values()))
near_iso_dict = {}
for combo in combos:
    if hetero_dict[combo] == min_heterovalency:
        near_iso_dict[combo] = combos[combo]
        
combos = near_iso_dict
choices = {cation : [] for cation in cations}
for cation in cations:
    for combo in combos:
        choices[cation].extend(combos[combo][cation])
        choices[cation] = list(set(choices[cation]))
        
print(choices)

### PICK UP HERE ###

{'Cs': [1], 'Ag': [1], 'Bi': [3]}


In [ ]:
### choose the most likely charge-balanced combination
combos = bal_combos

# generate a dictionary of {cation : electronegativity} for help with assignment
chi_dict = {}
with open('electronegativities.csv') as f:
    for line in f:
        line = line.split(',')
        if line[0] in cations:
            chi_dict[line[0]] = float(line[1][:-1])

# if only one charge-balanced combination exists, use it
if len(combos) == 1:
    ox_states = dict(zip(cations, combos[0]))
# if two combos exists and they are the reverse of one another
elif (len(combos) == 2) and (combos[0] == combos[1][::-1]):
    # assign the minimum oxidation state to the more electronegative cation
    min_ox = np.min(combos[0])
    max_ox = np.max(combos[1])
    epos_el = [el for el in cations if chi_dict[el] == np.min(list(chi_dict.values()))][0]
    eneg_el = [el for el in cations if el != epos_el][0]
    ox_states = {epos_el : max_ox,
                 eneg_el : min_ox}
else:
    # if one of the cations is probably 3+, let it be 3+
    if (cations[0] in plus_three) or (cations[1] in plus_three):
        if X == 'O':
            if (3,3) in combos:
                combo = (3,3)
                ox_states = dict(zip(ox_states, list(combo)))
    # else compare electronegativities - if 0.9 < chi1/chi2 < 1.1, minimize the oxidation state diff
    elif np.min(list(chi_dict.values())) > 0.9 * np.max(list(chi_dict.values())):
        diffs = [abs(combo[0] - combo[1]) for combo in combos]
        mindex = [idx for idx in range(len(diffs)) if diffs[idx] == np.min(diffs)]
        if len(mindex) == 1:
            mindex = mindex[0]
            combo = combos[mindex]
            ox_states = dict(zip(cations, combo))
        else:
            min_ox = np.min([combos[idx] for idx in mindex])
            max_ox = np.max([combos[idx] for idx in mindex])
            epos_el = [el for el in cations if chi_dict[el] == np.min(list(chi_dict.values()))][0]
            eneg_el = [el for el in cations if el != epos_el][0]
            ox_states = {epos_el : max_ox,
                         eneg_el : min_ox} 
    else:
        diffs = [abs(combo[0] - combo[1]) for combo in combos]
        maxdex = [idx for idx in range(len(diffs)) if diffs[idx] == np.max(diffs)]
        if len(maxdex) == 1:
            maxdex = maxdex[0]
            combo = combos[maxdex]
        else:
            min_ox = np.min([combos[idx] for idx in maxdex])
            max_ox = np.max([combos[idx] for idx in maxdex])
            epos_el = [el for el in cations if chi_dict[el] == np.min(list(chi_dict.values()))][0]
            eneg_el = [el for el in cations if el != epos_el][0]
            ox_states = {epos_el : max_ox,
                        eneg_el : min_ox}     
print('The electronegativities of %s = %.2f and %s = %.2f' 
      % (cations[0], chi_dict[cations[0]], cations[1], chi_dict[cations[1]]))
print('The assigned oxidation states are therefore %s = %.2f and %s = %.2f' 
      % (cations[0], ox_states[cations[0]], cations[1], ox_states[cations[1]]))

In [ ]:
### we know the oxidation states, but not which cation is A or B (yet)
### produce a dictionary of each cation as A or B
radii_dict = {}
for el in cations:
    tmp_dict = {}
    # get the oxidation state
    ox = ox_states[el]
    # get the coordination numbers for that cation by Shannon
    coords = list(ionic_radii_dict[el][ox].keys())
    # get the B CN as the one available nearest 6
    B_coords = [abs(coord - 6) for coord in coords]
    mindex = [idx for idx in range(len(B_coords)) if B_coords[idx] == np.min(B_coords)][0]
    B_coord = coords[mindex]
    # get the A CN as the one available nearest 12
    A_coords = [abs(coord - 12) for coord in coords]
    mindex = [idx for idx in range(len(A_coords)) if A_coords[idx] == np.min(A_coords)][0]
    A_coord = coords[mindex]
    # produce the equivalent B-site and A-site radii
    B_rad = ionic_radii_dict[el][ox][B_coord]['only_spin']
    A_rad = ionic_radii_dict[el][ox][A_coord]['only_spin']
    tmp_dict['A_rad'] = A_rad
    tmp_dict['B_rad'] = B_rad
    radii_dict[el] = tmp_dict

for el in cations:
    print('The radius of %s on the A site would be %.2f Angstrom' % (el, radii_dict[el]['A_rad']))
    print('The radius of %s on the B site would be %.2f Angstrom' % (el, radii_dict[el]['B_rad']))

In [ ]:
### determine A and B, where A is the larger cation


el1 = list(radii_dict.keys())[0]
el2 = list(radii_dict.keys())[1]

if (radii_dict[el1]['A_rad'] > radii_dict[el2]['B_rad']) and (radii_dict[el1]['B_rad'] > radii_dict[el2]['A_rad']):
    pred_A = el1
elif (radii_dict[el1]['A_rad'] < radii_dict[el2]['B_rad']) and (radii_dict[el1]['B_rad'] < radii_dict[el2]['A_rad']):
    pred_A = el2
elif (radii_dict[el1]['A_rad'] > radii_dict[el2]['A_rad']) and (radii_dict[el1]['B_rad'] > radii_dict[el2]['B_rad']):
    pred_A = el1
elif (radii_dict[el1]['A_rad'] < radii_dict[el2]['A_rad']) and (radii_dict[el1]['B_rad'] < radii_dict[el2]['B_rad']):
    pred_A = el2
elif (radii_dict[el1]['B_rad'] > radii_dict[el2]['B_rad']):
    pred_A = el1
elif (radii_dict[el1]['B_rad'] < radii_dict[el2]['B_rad']):
    pred_A = el2
elif (radii_dict[el1]['A_rad'] > radii_dict[el2]['A_rad']):
    pred_A = el1
elif (radii_dict[el1]['A_rad'] < radii_dict[el2]['A_rad']):
    pred_A = el2  
else:
    # if the A and B radii are the same for both elements, choose the more oxidized element
    if ox_dict[el1] < ox_dict[el2]:
        pred_A = el1
    else:
        # if the elements have the same radii and oxidation state, choose at random
        pred_A = el2
        
pred_B = [el for el in cations if el != pred_A][0]

print('%s is predicted to be A site with oxidation state = %i and radius = %.2f' 
       % (pred_A, ox_states[pred_A], radii_dict[pred_A]['A_rad']))
print('%s is predicted to be B site with oxidation state = %i and radius = %.2f' 
       % (pred_B, ox_states[pred_B], radii_dict[pred_B]['B_rad']))

In [ ]:
### make classification using tau

nA = ox_states[pred_A]
rA = radii_dict[pred_A]['A_rad']
rB = radii_dict[pred_B]['B_rad']
rX = ionic_radii_dict[X][X_ox_dict[X]][6]['only_spin']

tau = rX/rB - nA * (nA - (rA/rB)/np.log(rA/rB))

print('tau = %.2f which is %s 4.18, so %s is predicted %s' 
      % (tau, '<' if tau < 4.18 else '>', CCX3, 'perovskite' if tau < 4.18 else 'nonperovskite'))